In [1]:
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

Testing the functions of the module mapping_fits_objects.py instead.

In [2]:
if IN_COLAB:
    root_path = "/content/"+iqa_root
else:
    root_path = "/home/mike/git/computational_astro/astro_iqa"
src_path = os.path.join(root_path, "src")

print(os.listdir(src_path))
sys.path.append(os.path.abspath(src_path))

from modeling.mapping_fits_objects import *

data_path = os.path.join(root_path, "data/for_modeling")
catalog_name = "objects_catalog_cadc2.parquet.gz"

['data_acquisition_understanding', 'deployment', 'modeling', 'Readme.md']


In [3]:
catalog_file = os.path.join(data_path, catalog_name)

columns = ["OBJECT_ID", "FITS_ID", "CCD_ID", "ISO0", "BACKGROUND", "ELLIPTICITY", "ELONGATION", "CLASS_STAR", "FLAGS", "EXPTIME"]
catalog_df = read_catalog(catalog_file, columns)


In [4]:
annotations_file = os.path.join(data_path, "map_images_labels_cadc2.json")
annotations = read_annotations(annotations_file)



In [5]:
annotations_df = transform_annotations_to_df(annotations)
merged_df = merge_annotations_with_catalog(catalog_df, annotations_df)

0              GOOD
1               BGP
2              GOOD
3              GOOD
4              GOOD
5              GOOD
6                BT
7                BT
8                BT
9              GOOD
10             GOOD
11             GOOD
12         B_SEEING
13             GOOD
14             GOOD
15             GOOD
16             GOOD
17              RBT
18         B_SEEING
19              RBT
20              RBT
21               BT
22    BGP, B_SEEING
23     B_SEEING, BT
24     BT, B_SEEING
Name: Label, dtype: object
0         GOOD
1          BGP
2         GOOD
3         GOOD
4         GOOD
5         GOOD
6           BT
7           BT
8           BT
9         GOOD
10        GOOD
11        GOOD
12    B_SEEING
13        GOOD
14        GOOD
15        GOOD
16        GOOD
17         RBT
18    B_SEEING
19         RBT
20         RBT
21          BT
22         BGP
23    B_SEEING
24          BT
Name: Label1, dtype: object


In [ ]:
output_file = os.path.join(data_path, "objects_catalog_cadc3.parquet.gz")
save_to_parquet(merged_df, output_file)